In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
import torch.optim.lr_scheduler as lr_scheduler

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("./drive/My Drive/workspaces")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFaRKBCaiV4qGqvzNTrj4kIDxnaw0i6ZoCtYN6bkrbN5ugin4pno20
Mounted at /content/drive


In [3]:
# 超参数设置
EPOCH = 50   #遍历数据集次数
pre_epoch = 0  # 定义已经遍历数据集的次数
BATCH_SIZE = 128      #批处理尺寸(batch_size)
LR = 0.1        #学习率
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), #R,G,B每层的归一化用到的均值和方差
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='nas/cifar10', train=True, download=False, transform=transform_train) #训练数据集
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)   #生成一个个batch进行批训练，组成batch的时候顺序打乱取

testset = torchvision.datasets.CIFAR10(root='nas/cifar10', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
# Cifar-10的标签

net = ResNet18().to(device)
#criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
scheduler = lr_scheduler.StepLR(optimizer, step_size=15,gamma=0.1)
net.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (layer1): Sequential(
    (0): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inp

In [4]:
# 训练
for epoch in range(EPOCH):
    if scheduler:
      scheduler.step()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        net.train()
        # 前向传播
        outputs = net(inputs)
        # 计算损失函数
        #loss = criterion(outputs, labels)
        loss = F.cross_entropy(outputs, labels)
        # 清空上一轮梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
 
    print('epoch{} loss:{:.4f}'.format(epoch+1, loss.item()))
    net.eval()                                   #测试模式
    with torch.no_grad():             
      total_correct = 0                           #预测正确的个数
      total_num = 0
      for i, data in enumerate(testloader): 
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        out = net(inputs)
        _, predicted = torch.max(out.data, 1)
        total_num += labels.size(0)
        total_correct += (predicted == labels).sum().item()                            
      print('10000测试图像 准确率:{:.4f}%'.format(100 * total_correct / total_num)) 

print("Finished Traning")

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch1 loss:1.7551
10000测试图像 准确率:37.2100%
epoch2 loss:1.4598
10000测试图像 准确率:51.6800%
epoch3 loss:0.9833
10000测试图像 准确率:59.2700%
epoch4 loss:1.0116
10000测试图像 准确率:63.8800%
epoch5 loss:0.8491
10000测试图像 准确率:67.9800%
epoch6 loss:0.8249
10000测试图像 准确率:75.5100%
epoch7 loss:0.6565
10000测试图像 准确率:68.9100%
epoch8 loss:0.5601
10000测试图像 准确率:73.5700%
epoch9 loss:0.7286
10000测试图像 准确率:75.9100%
epoch10 loss:0.7084
10000测试图像 准确率:74.0800%
epoch11 loss:0.5131
10000测试图像 准确率:74.3600%
epoch12 loss:0.5572
10000测试图像 准确率:80.2800%
epoch13 loss:0.4434
10000测试图像 准确率:76.3700%
epoch14 loss:0.5036
10000测试图像 准确率:79.5300%
epoch15 loss:0.4483
10000测试图像 准确率:89.5200%
epoch16 loss:0.3143
10000测试图像 准确率:89.8700%
epoch17 loss:0.1216
10000测试图像 准确率:90.1900%
epoch18 loss:0.2239
10000测试图像 准确率:90.5000%
epoch19 loss:0.1323
10000测试图像 准确率:90.7000%
epoch20 loss:0.1120
10000测试图像 准确率:90.7400%
epoch21 loss:0.1959
10000测试图像 准确率:91.1700%
epoch22 loss:0.1968
10000测试图像 准确率:90.8000%
epoch23 loss:0.1634
10000测试图像 准确率:90.9700%
epoch24 loss:0.0898


参照网上的写法说是可以达到90%以上，但实际并没有实现
个人理解: 在原始的resnet上，做了些优化：
1.原始的7*7的卷积核改为3*3，去掉了最大池化部分，这两部分的修改猜测是因为输入原始图片32*32过小，所以去掉了一些特征提取和降维的处理
2.另外对原始数据做了一些预处理，进行随机翻转，正则化，都有助于提升准备率.
3.调整LR，EPOCH，BATCH_SIZE等
4.问题：训练太慢了，有啥好办法，而且容易断线

In [2]:
!/opt/bin/nvidia-smi

/bin/bash: /opt/bin/nvidia-smi: No such file or directory
